<a href="https://colab.research.google.com/github/dgalassi99/OOP_learning/blob/main/ipynbs/04_Py_OOP_MoneyTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Budget and Expense Tracker

Track income, expenses, and budgets over time. Later, analyze savings, trends, and budgeting performance. Eventually display this via a dashboard.

## Base Structure Implementation

### Transaction - data container for each entry

In [ ]:
from datetime import datetime

class Transaction:

  def __init__(self,date,amount,category,description=''):
    self.amount = amount
    self.date = datetime.strptime(date, '%d-%m-%Y')
    self.category = category
    self.description = description

  def __repr__(self):
    return f'Transaction(amount={self.amount}, date={self.date}, category={self.category}, description={self.description})'

In [ ]:
trs = Transaction('04-05-2025',100,'food','testing the class __repr__ method')
trs.__repr__()

'Transaction(amount=100, date=2025-05-04 00:00:00, category=food, description=testing the class __repr__ method)'

### Budget - category limit

In [ ]:
class Budget:

  def __init__(self,category,limit):
    self.category = category
    self.limit = limit

  def __repr__(self):
        return f"{self.category} Budget: {self.limit:.2f}"

In [ ]:
bg = Budget('food',100)
bg.__repr__()

'food Budget: 100.00'

### Expense Tracker - the brain of the tracker

In [ ]:
class Tracker:

  def __init__(self):
    self.transactions = [] #list of transaction
    self.budgets = {} #dict of budget- category:limit

#method to add a transaction
  def add_transaction(self,transaction):
    self.transactions.append(transaction) #add transaction to the transaction list

#method to filter transaction based on category, start and end date
  def filter_transactions(self, start_date=None, end_date=None, category=None):
    from datetime import datetime
    filtered = [] #create a list
    for txn in self.transactions: #loop inside the transaction
        txn_date = txn.date
        if category and txn.category != category: #if not matching skip
            continue
        if start_date:
            start_date_dt = datetime.strptime(start_date, '%d-%m-%Y')
            if txn_date < start_date_dt:
                continue
        if end_date:
            end_date_dt = datetime.strptime(end_date, '%d-%m-%Y')
            if txn_date > end_date_dt:
                continue
        filtered.append(txn)
    return filtered

#getter to get a transaction
  def get_transactions_log(self, start_date=None, end_date=None, category=None):
      import pandas as pd
    # get filtered transactions using the filter_transactions method
      filtered = self.filter_transactions(start_date, end_date, category)
      if not filtered:  # if no transactions, return an empty DataFrame
        return pd.DataFrame(columns=["date", "amount", "category"])
      #create a df with the filtered transactions to have a dataframe style log
      data = []
      for txn in filtered:
          data.append({'date': txn.date, 'amount': txn.amount, 'category': txn.category})
      return pd.DataFrame(data)

#method to add a budget using Budget class
  def add_budget(self,category,limit):
    self.budgets[category] = Budget(category,limit)

#getter to get a budget
  def get_budgets(self,category):
    return self.budgets.get(category,None)

#getter to get total by a given category
  def get_summary(self, start_date=None, end_date=None, category=None):
    #get filtered transactions using the filter_transactions method
    filtered = self.filter_transactions(start_date, end_date, category)
    #calcualte some summaties
    income = sum(txn.amount for txn in filtered if txn.amount > 0)
    expenses = sum(txn.amount for txn in filtered if txn.amount < 0)
    net = income + expenses  # expenses is negative, hence added with +
    return {'income': income, 'expenses': expenses, 'net': net}


### Test #1

In [ ]:
tracker = Tracker()

#add a few transactions
tracker.add_transaction(Transaction('01-04-2025',-50, 'Groceries'))
tracker.add_transaction(Transaction('02-04-2025',-20, 'Transport'))
tracker.add_transaction(Transaction('01-04-2025',2000, 'Salary'))
tracker.add_transaction(Transaction('10-04-2025',-100, 'Groceries'))
tracker.add_transaction(Transaction('15-04-2025',-30, 'Transport'))

#add some budgets
tracker.add_budget('Groceries', 300)
tracker.add_budget('Transport', 100)

#full summary
print("Full Summary:")
print(tracker.get_summary())

#summary for groceries only
print("\nGroceries Summary:")
print(tracker.get_summary(category='Groceries'))

#summary for a date range
print("\nSummary between 2025-04-01 and 2025-04-10:")
print(tracker.get_summary(start_date='01-04-2025', end_date='10-04-2025'))

#get all transactions
print("\nAll transactions so far")
tracker.get_transactions_log()

Full Summary:
{'income': 2000, 'expenses': -200, 'net': 1800}

Groceries Summary:
{'income': 0, 'expenses': -150, 'net': -150}

Summary between 2025-04-01 and 2025-04-10:
{'income': 2000, 'expenses': -170, 'net': 1830}

All transactions so far


,date,amount,category
0,2025-04-01,-50,Groceries
1,2025-04-02,-20,Transport
2,2025-04-01,2000,Salary
3,2025-04-10,-100,Groceries
4,2025-04-15,-30,Transport
